# 320. Custom Dataset을 이용한 Hugging Face BERT model Fine Tuning

- NAVER Movie review dataset을 이용하여 transformers BERT model을 fine tuning  

- Pytorch 와 Trainer를 이용한 Fine Tuning (Pytorch version이 Tensorflow 보다 안정적)

In [1]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch.nn.functional as F
import tensorrt as tr
import tensorflow as tf
import pandas as pd

/home/paradeigma/workspace/python/nlp_module/nlp_module_3_9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 19:04:26.652204: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 19:04:26.743773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 19:04:27.569685: W tensorflow/compiler/tf2tensorr

In [2]:
# DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt",
#                      "https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_train.txt")
# DATA_TEST_PATH = tf.keras.utils.get_file("ratings_test.txt",
#                     "https://raw.github.com/ironmanciti/Infran_NLP/master/data/naver_movie/ratings_test.txt")

DATA_TRAIN_PATH =  "../data/ratings_train.txt"
DATA_TEST_PATH = "../data/ratings_test.txt"

### Train Set

In [3]:
train_data = pd.read_csv(DATA_TRAIN_PATH, delimiter='\t')
print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
train_data.dropna(inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


### Test Set

In [5]:
test_data = pd.read_csv(DATA_TEST_PATH, delimiter='\t')
print(test_data.shape)
test_data.head()

(50000, 3)


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
test_data.dropna(inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 훈련 시간 단축을 위해 1/10 의 data 만 sampling - 6분 소요

In [7]:
df_train = train_data.sample(n=15000, random_state=1)
df_test = test_data.sample(n=5000, random_state=1)
print(df_train.shape)
print(df_test.shape)

(15000, 3)
(5000, 3)


In [8]:
df_train['label'].value_counts()

label
0    7524
1    7476
Name: count, dtype: int64

In [9]:
X_train = df_train['document'].values.tolist()
y_train = df_train['label'].values.tolist()

X_test = df_test['document'].values.tolist()
y_test = df_test['label'].values.tolist()

## pre-trained bert model 호출
### tokenizer 호출
- 토큰화 처리를 합니다. bert 다국어 version 용의 pre-trained tokenizer 를 불러옵니다.

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

pre-trained tokenizer 를 이용하여 train set 과 test set 을 token 화 합니다.

- Input IDs : 토큰 인덱스, 모델에서 입력으로 사용할 시퀀스를 구축하는 토큰의 숫자 표현
- Token Type IDs : 한 쌍의 문장 또는 질문 답변에 대한 분류 시 사용  
- attention mask : `1`은 주목해야 하는 값을 나타내고 `0`은 패딩된 값을 나타냅니다.  
```
[CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
ex) [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
```

In [11]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [12]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0]) # PADDING이 된 것은 0  아니면 1
print(train_encodings['token_type_ids'][0]) # 문장이 연달아 있을 때 0, 1로 구분분

[101, 113, 9926, 34907, 20626, 58931, 24974, 122, 114, 9532, 25503, 12030, 28911, 9367, 19855, 47869, 9682, 9634, 21386, 136, 8924, 11261, 119351, 12605, 20308, 12453, 117, 9792, 73352, 21876, 20173, 9294, 36553, 11287, 52560, 9391, 11664, 9640, 18784, 12030, 12508, 9304, 12508, 19709, 9684, 52560, 10892, 8932, 118651, 14523, 48549, 119, 8905, 119377, 11102, 117, 9604, 78123, 11102, 117, 9684, 89523, 42769, 15387, 9792, 73352, 21876, 20173, 47058, 8982, 28188, 11664, 9294, 36553, 11287, 52560, 9597, 10530, 19709, 9792, 73352, 21876, 100698, 11018, 9670, 14871, 15387, 9637, 12945, 22333, 43022, 113, 9069, 18227, 114, 63783, 9641, 42337, 14801, 119, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Convert encodings to Tensors

- 레이블과 인코딩을 Dataset 개체로 변환합니다. Pytorch를 이용합니다.  

- PyTorch에서 이것은 `torch.utils.data.Dataset` 객체를 하고 `__len__` 및 `__getitem__`을 구현하여 수행됩니다.

- TensorFlow에서는 입력 인코딩과 레이블을 `from_tensor_slices` 생성자 메서드에 전달합니다. (불안정)

In [14]:
# torch dataset을 생성하여 imdb에 공급하는 역할
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = IMDbDataset(train_encodings, y_train)
test_dataset = IMDbDataset(test_encodings, y_test)

이제 데이터 세트가 준비되었으므로 🤗 `Trainer` 또는 기본 PyTorch/TensorFlow를 사용하여 모델을 미세 조정할 수 있습니다. [training](https://huggingface.co/transformers/training.html)을 참조하세요.

- Training warmup steps :  

    - 이는 일반적으로 설정된 수의 훈련 단계(워밍업 단계)에 대해 매우 낮은 학습률을 사용한다는 것을 의미합니다. 워밍업 단계 후에 "일반" 학습률 또는 학습률 스케줄러를 사용합니다. 또한 워밍업 단계 수에 따라 학습률을 점진적으로 높일 수 있습니다.

- weight_decay : 가중치 감쇠. L2 regularization

In [15]:
training_args = TrainingArguments(
    output_dir='../results',               # output 저장 directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size per device during evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # weight decay 강도
    logging_dir='../logs',            # log 저장 directory
    logging_steps=10,
)

### model Train

In [16]:
import time

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

s = time.time()

trainer.train()

/home/paradeigma/workspace/python/nlp_module/nlp_module_3_9/lib/python3.9/site-packages/transformers/modeling_utils.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ret

Step,Training Loss
10,0.691500
20,0.687300
30,0.690900
40,0.699200
50,0.692000
60,0.679900
70,0.653900
80,0.694100
90,0.669900
100,0.637600


Saving model checkpoint to ../results/checkpoint-500
Configuration saved in ../results/checkpoint-500/config.json
Model weights saved in ../results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ../results/checkpoint-1000
Configuration saved in ../results/checkpoint-1000/config.json
Model weights saved in ../results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ../results/checkpoint-1500
Configuration saved in ../results/checkpoint-1500/config.json
Model weights saved in ../results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1876, training_loss=0.4459348225644402, metrics={'train_runtime': 489.732, 'train_samples_per_second': 61.258, 'train_steps_per_second': 3.831, 'total_flos': 1896249598200000.0, 'train_loss': 0.4459348225644402, 'epoch': 2.0})

In [17]:
print("경과 시간 : {:.2f}분".format((time.time() - s)/60))

경과 시간 : 8.18분


In [18]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'eval_loss': 0.43356195092201233,
 'eval_runtime': 21.1949,
 'eval_samples_per_second': 235.906,
 'eval_steps_per_second': 14.768,
 'epoch': 2.0}

In [19]:
prediction = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 5000
  Batch size = 16


fine-tuned model 은 logit 을 return

In [20]:
trainer.model.classifier

Linear(in_features=768, out_features=2, bias=True)

In [21]:
y_logit = torch.tensor(prediction[0])
y_logit[:10]

tensor([[ 1.4821, -1.2058],
        [ 1.8418, -1.4673],
        [ 1.5789, -1.3152],
        [ 2.3567, -1.9517],
        [-2.0725,  1.7273],
        [ 2.3238, -1.8471],
        [ 0.0599,  0.0890],
        [ 2.2259, -1.7808],
        [-2.2371,  1.7241],
        [ 1.0616, -0.8287]])

In [22]:
y_pred = F.softmax(y_logit, dim=-1).argmax(axis=1).numpy()
print(list(y_pred[:30]))
print(y_test[:30])

[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]


In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(accuracy_score(y_test, y_pred))

cm=confusion_matrix(y_test, y_pred)
cm

0.8214


array([[2005,  490],
       [ 403, 2102]])

In [24]:
# x = '돈주고 보기에는 아까운 영화 ㅠㅠ...'
x = '정말 재미있는 영화'
tokenized = tokenizer([x], truncation=True, padding=True)
pred = trainer.predict(IMDbDataset(tokenized))

logit = torch.tensor(pred[0])
result = F.softmax(logit, dim=-1).argmax(1).numpy()
"긍정" if result == 1 else "부정"

***** Running Prediction *****
  Num examples = 1
  Batch size = 16


'긍정'

# Next Step
20 만개 전체 dataset으로 fine tuning